### 라이브러리 임포트

In [1]:
import sys
print(sys.executable)  # 반드시 ...\envs\geo_hotspot2\python.exe 여야 함


c:\Users\wonny\anaconda3\python.exe


In [2]:
import time
import math

In [3]:
import numpy as np

In [9]:
import pandas as pd

In [5]:
import re

In [6]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [7]:
import os, json, textwrap
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

In [10]:
from libpysal.weights import KNN, DistanceBand

In [11]:
from difflib import SequenceMatcher

In [12]:
from tqdm import tqdm

### 경기도 상권 매출 데이터 수집 

In [ ]:
BASE_URL = "https://openapi.gg.go.kr/TBGGESTDEVALLSTM"
API_KEY  = "" 

P_SIZE   = 1000
SLEEP    = 0.2  # 각 호출 사이 대기(초) - 서버 과부하/차단 예방

def call_api(page: int, psize: int = P_SIZE) -> dict:
    """단일 페이지 호출"""
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "pIndex": page,
        "pSize": psize
    }
    r = requests.get(BASE_URL, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def extract_total_count(payload: dict) -> int:
    """응답에서 전체 건수 추출 (대/소문자 섞임 방지)"""
    root = payload.get("TBGGESTDEVALLSTM", [])
    if not root:
        raise ValueError("응답에 'TBGGESTDEVALLSTM' 키가 없습니다.")
    head = root[0].get("head", [])
    if not head:
        raise ValueError("응답에 'head' 블록이 없습니다.")
    # 보통 head[0]에 list_total_count, head[1].RESULT에 코드/메시지
    h0 = head[0]
    for k in ("list_total_count", "LIST_TOTAL_COUNT"):
        if k in h0:
            return int(h0[k])
    # 간혹 다른 위치에 있을 가능성까지 대비
    for item in head:
        for k in ("list_total_count", "LIST_TOTAL_COUNT"):
            if k in item:
                return int(item[k])
    raise ValueError("전체 건수를 찾지 못했습니다.")

def extract_rows(payload: dict) -> list:
    """응답에서 데이터 행(row) 리스트 추출"""
    root = payload.get("TBGGESTDEVALLSTM", [])
    if len(root) < 2:
        return []  # 데이터 없는 페이지일 수 있음
    rows = root[1].get("row", [])
    return rows or []

def fetch_all(psize: int = P_SIZE, sleep_sec: float = SLEEP) -> pd.DataFrame:
    """전체 페이지 크롤링 → DataFrame 반환"""
    # 1) 첫 페이지로 전체 건수 확인
    first = call_api(page=1, psize=psize)
    total_count = extract_total_count(first)
    total_pages = max(1, math.ceil(total_count / psize))
    print(f"[INFO] 총 건수: {total_count:,} / 총 페이지: {total_pages}")

    # 2) 첫 페이지 rows 적재
    all_rows = extract_rows(first)

    # 3) 2페이지부터 반복 수집
    for page in range(2, total_pages + 1):
        try:
            payload = call_api(page=page, psize=psize)
            rows = extract_rows(payload)
            all_rows.extend(rows)
        except Exception as e:
            print(f"[WARN] pIndex={page} 수집 실패: {e}")
        time.sleep(sleep_sec)

    # 4) DataFrame 변환
    df = pd.DataFrame(all_rows)
    # (선택) 컬럼 순서 정리: 문서에 나온 대표 컬럼들이 있으면 앞으로 배치
    preferred_cols = [
        "STD_YY","QU_NM","DIV","BIZDIST_NM","CLASS_CD","CLASS_CD_NM","AMT","NOC",
        "API_VERSION","CODE","MESSAGE"
    ]
    cols = [c for c in preferred_cols if c in df.columns] + [c for c in df.columns if c not in preferred_cols]
    df = df[cols]
    return df

if __name__ == "__main__":
    df_all = fetch_all(psize=1000, sleep_sec=0.2)
    print(df_all.shape)
    print(df_all.head())

    # CSV 저장 (파일명은 자유롭게 변경)
    out_path = "경기도골목상권매출.csv"
    df_all.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[DONE] 저장완료: {out_path}")


[INFO] 총 건수: 69,194 / 총 페이지: 70


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("경기도골목상권매출.csv", encoding="utf-8-sig")

In [15]:
df

,STD_YY,QU_NM,DIV,BIZDIST_NM,CLASS_CD,CLASS_CD_NM,AMT,NOC
0,2024,3,1143,여양로,47591,전기용품 및 조명장치 소매업,2352809,36
1,2024,3,1143,여양로,47592,"주방용품 및 가정용 유리, 요업제품 소매업",6969499,5
2,2024,3,1143,여양로,47611,"서적, 신문 및 잡지류 소매업",45261,1
3,2024,3,1143,여양로,47631,운동 및 경기용품 소매업,5416628,20
4,2024,3,1143,여양로,47632,자전거 및 기타 운송장비 소매업,4762640,90
...,...,...,...,...,...,...,...,...
69189,2024,3,1648,올레플라자시흥점,47591,전기용품 및 조명장치 소매업,12616929,202
69190,2024,3,1648,올레플라자시흥점,47520,가구 소매업,2032093,1
69191,2024,3,1648,올레플라자시흥점,47511,철물 및 난방용구 소매업,62150740,239
69192,2024,3,1648,올레플라자시흥점,47422,의복 액세서리 및 모조 장신구 소매업,33922877,522


In [16]:
# 필요없는 칼럼 제거
df.drop(columns=["STD_YY", "QU_NM", "DIV", "CLASS_CD_NM"], inplace=True)

# 결과 확인
print(df.head())
print(df.columns)  # 남은 컬럼 확인


  BIZDIST_NM  CLASS_CD      AMT  NOC
0        여양로     47591  2352809   36
1        여양로     47592  6969499    5
2        여양로     47611    45261    1
3        여양로     47631  5416628   20
4        여양로     47632  4762640   90
Index(['BIZDIST_NM', 'CLASS_CD', 'AMT', 'NOC'], dtype='object')


### 위경도 매핑

In [ ]:
# Kakao API 키 (추천: 환경변수로 관리)
KAKAO_REST_KEY = os.getenv("KAKAO_REST_KEY", "")
KAKAO_JS_KEY   = os.getenv("KAKAO_JS_KEY",   "")
HEADERS = {"Authorization": f"KakaoAK {KAKAO_REST_KEY}"}

In [ ]:
# 경기도 전체 bbox (W,S,E,N)
GG_RECT = "126.60,36.90,127.80,38.30"

In [ ]:
# ✅ 체크포인트/캐시/슬립
CHECKPOINT_EVERY = 50
CHECKPOINT_PATH  = "map_progress.csv"     # 진행상황 snapshot
CACHE_PATH       = "map_cache.json"       # name -> {lon,lat,payload}
FAILED_PATH      = "failed_names.csv"
SLEEP_SEC        = 0.35

In [ ]:
# ============================================
# 🔧 1) 유틸
# ============================================
def load_cache(path: str) -> dict:
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return {}
    return {}

def save_cache(path: str, cache: dict):
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False)
    os.replace(tmp, path)

def within_rect(lon: float, lat: float, rect: str = GG_RECT) -> bool:
    W, S, E, N = map(float, rect.split(","))
    return (W <= lon <= E) and (S <= lat <= N)

def _normalize_text(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", "", s)
    s = re.sub(r"(본점|지점|점)$", "", s)
    s = re.sub(r"[-_·•ㆍ]", "", s)
    return s

def _is_road_like(name: str) -> bool:
    return bool(re.search(r"(길|로|대로)$", name))

def _extract_city(addr: str) -> str | None:
    if not addr: return None
    parts = addr.split()
    if len(parts) >= 2 and parts[0].startswith("경기"):
        return parts[1]
    return None

def guess_category(name: str) -> str | None:
    s = str(name)
    if re.search(r"(초등학교|중학교|고등학교|학교)", s): return "SC4"
    if re.search(r"(주민센터|구청|시청|행정복지센터|동사무소|우체국|우편취급국|취급국|소방서|경찰서)", s): return "PO3"
    if re.search(r"(보건소|병원|의원|치과|한의원)", s): return "HP8"
    return None

def is_in_gyeonggi(place: dict) -> bool:
    for k in ("road_address_name", "address_name"):
        v = (place.get(k) or "").strip()
        if re.match(r"^경기(도)?(\s|$)", v):
            return True
    return False


# ============================================
# 🌐 2) Kakao API 래퍼
# ============================================
def kakao_keyword_search(query: str, rect: str = GG_RECT, category: str | None = None, size: int = 10) -> list[dict]:
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    params = {"query": query, "rect": rect, "size": size}
    if category:
        params["category_group_code"] = category
    for attempt in range(3):
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=5)
            if r.status_code == 200:
                return r.json().get("documents", [])
        except requests.RequestException:
            pass
        time.sleep(0.25 + 0.25*attempt)
    return []

def kakao_address_search(query: str, size: int = 10) -> list[dict]:
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": query, "size": size}
    for attempt in range(2):
        try:
            r = requests.get(url, headers=HEADERS, params=params, timeout=5)
            if r.status_code == 200:
                return r.json().get("documents", [])
        except requests.RequestException:
            pass
        time.sleep(0.25 + 0.25*attempt)
    return []


# ============================================
# 🧠 3) 실패패턴 교정(정규화) + 동의어 보정
# ============================================
ALIAS = {
    # 역 출구 언더스코어
    "부천시청역_1번출구": "부천시청역 1번출구",
    "부천시청역_3번출구": "부천시청역 3번출구",
    "부천시청역_4번출구": "부천시청역 4번출구",
    "수원시청역_1번출구": "수원시청역 1번출구",
    "수원시청역_7번출구": "수원시청역 7번출구",
    # 도로명+번길 공백
    "경의로146번길": "경의로 146번길",
    "서암로6번길":  "서암로 6번길",
    "운중로267번길": "운중로 267번길",
    # 기타 오타/띄어쓰기
    "뉴프린스관광호텔": "프린스관광호텔",
    "롯데애비뉴엘안산점": "롯데 에비뉴엘 안산점",
    "롯데애비뉴엘일산점": "롯데 에비뉴엘 일산점",
    "롯데마트빅마켓킨텍스점": "롯데마트 빅마켓 킨텍스점",
    "홈플러스의전부점": "홈플러스 의정부점",
}

SYNONYM_RULES = [
    (r"정류소$", "정류장"),
    (r"공용버스터미널$", "버스터미널"),
    (r"역_?(\d+)번출구$", r"역 \1번출구"),
]

def insert_spaces_for_admin(s: str) -> str:
    s = re.sub(r"^([가-힣]+)([가-힣]+(동|읍|면|리))우체국$",     r"\1 \2 우체국", s)
    s = re.sub(r"^([가-힣]+)([가-힣]+(동|읍|면|리))정류장$",     r"\1 \2 정류장", s)
    s = re.sub(r"^([가-힣]+)([가-힣]+(동|읍|면|리))버스터미널$", r"\1 \2 버스터미널", s)
    s = re.sub(r"^([가-힣]+)([가-힣]+(동|읍|면|리))역(\s*\d+번출구)?$", r"\1 \2 역\4", s)
    return s

def space_road_bn(s: str) -> str:
    s = re.sub(r"(로|길|대로)(\d+)번길$", r"\1 \2번길", s)
    s = re.sub(r"(로|길|대로)(\d+)$",     r"\1 \2", s)
    return s

def normalize_candidates(name: str) -> list[str]:
    base = name.strip()
    outs = []
    def push(x): 
        x = x.strip()
        if x and x not in outs: outs.append(x)

    if base in ALIAS: push(ALIAS[base])

    n1 = re.sub(r"_[0-9]+$", "", base)                          # 접미 _숫자 제거
    n2 = re.sub(r"역_([0-9]+)번출구$", r"역 \1번출구", n1)        # 역_출구 → 공백
    n3 = space_road_bn(n2)                                       # 도로명+번길 공백
    n4 = insert_spaces_for_admin(n3)                             # 행정동/읍/면/리 띄어쓰기
    n5 = n4
    for pat, repl in SYNONYM_RULES: n5 = re.sub(pat, repl, n5)   # 동의어

    # 우체국 -> 우편취급국/취급국 변형도 시도
    if n5.endswith("우체국"):
        push(n5.replace("우체국", "우편취급국"))
        push(n5.replace("우체국", "취급국"))

    # 지역 힌트 추가 후보
    for x in [base, n1, n2, n3, n4, n5]:
        push(x)
    push("경기 " + n5)
    push("경기도 " + n5)
    return outs


# ============================================
# 🧭 4) 핵심: 지오코딩 함수(보정/다중후보/도로명엄격)
# ============================================
STRICT_ROAD_NAMES = {"벚꽃길"}
ROAD_SCORE_THRESHOLD   = 75
STRICT_SCORE_THRESHOLD = 85

def geocode_one(name: str, retry_sleep: float = 0.25):
    name = str(name).strip()
    if not name:
        return None, None, None

    queries = normalize_candidates(name)
    for q in queries:
        cat = guess_category(q)
        road_like  = _is_road_like(q)
        strict_mode = q in STRICT_ROAD_NAMES

        # 1) 키워드 검색
        docs = kakao_keyword_search(q, rect=GG_RECT, category=cat, size=10)
        if not docs:
            # 도로명일 때 주소검색 fallback
            if road_like:
                addrs = kakao_address_search(q, size=10)
                addr_docs = []
                for a in addrs:
                    region1 = (a.get("address", {}) or {}).get("region_1depth_name") or \
                              (a.get("road_address", {}) or {}).get("region_1depth_name")
                    x = a.get("x"); y = a.get("y")
                    if region1 == "경기" and x and y and within_rect(float(x), float(y), GG_RECT):
                        addr_docs.append({
                            "x": x, "y": y, "place_name": q,
                            "address_name": (a.get("address", {}) or {}).get("address_name"),
                            "road_address_name": (a.get("road_address", {}) or {}).get("address_name")
                        })
                if addr_docs:
                    best = addr_docs[0]
                    return float(best["x"]), float(best["y"]), best
            time.sleep(retry_sleep)
            continue

        # 2) 경기도 우선
        candidates = [d for d in docs if is_in_gyeonggi(d)] or docs
        if not candidates:
            time.sleep(retry_sleep); continue

        # 3) 도로명이면 road_address 포함 후보 우선
        if road_like:
            road_pat = re.compile(rf'(?<![가-힣0-9]){re.escape(q)}(?![가-힣])')
            c_road = [d for d in candidates if road_pat.search(d.get("road_address_name") or "")]
            if c_road: 
                candidates = c_road
            else:
                c_addr = [d for d in candidates if road_pat.search(d.get("address_name") or "")]
                if c_addr: 
                    candidates = c_addr

        # 4) 스코어
        n_q = _normalize_text(q)
        def score(d: dict) -> int:
            pname = (d.get("place_name") or "").strip()
            n_p = _normalize_text(pname)
            ra = (d.get("road_address_name") or "")
            aa = (d.get("address_name") or "")
            s = 0
            if n_p == n_q: s = 100
            elif n_q in n_p or n_p in n_q: s = 80
            else: s = int(SequenceMatcher(None, n_p, n_q).ratio() * 75)

            # '경기' 접두 보너스
            if re.match(r"^경기(도)?(\s|$)", ra) or re.match(r"^경기(도)?(\s|$)", aa):
                s += 5

            # 도로명 가중치
            if road_like:
                road_pat = re.compile(rf'(?<![가-힣0-9]){re.escape(q)}(?![가-힣])')
                in_road = bool(road_pat.search(ra))
                in_addr = bool(road_pat.search(aa))
                if in_road: s += 25
                elif in_addr: s += 5; s -= 10
                else: s -= 30

            # 키워드 보너스(POI류)
            key_bonus = 0
            if re.search(r"우체국|우편취급국|취급국", q): key_bonus += 10
            if re.search(r"정류장|버스터미널|터미널", q): key_bonus += 8
            if re.search(r"역\s*\d+번출구|역$", q): key_bonus += 6
            if re.search(r"시장", q): key_bonus += 5
            if re.search(r"호텔|마트|갤러리|전시관|에비뉴엘", q, re.I): key_bonus += 4
            s += key_bonus
            return s

        best = max(candidates, key=score)
        try:
            lon = float(best["x"]); lat = float(best["y"])
        except (KeyError, ValueError, TypeError):
            time.sleep(retry_sleep); continue

        # 5) 도로명 최종 검증/보정
        if road_like:
            road_pat = re.compile(rf'(?<![가-힣0-9]){re.escape(q)}(?![가-힣])')
            ra = (best.get("road_address_name") or "")
            sc = score(best)
            if q in STRICT_ROAD_NAMES:
                if not road_pat.search(ra) or sc < STRICT_SCORE_THRESHOLD:
                    time.sleep(retry_sleep); continue
            else:
                if sc < ROAD_SCORE_THRESHOLD:
                    time.sleep(retry_sleep); continue

            if not road_pat.search(ra):
                # address API로 보정 시도
                addrs = kakao_address_search(q, size=10)
                for a in addrs:
                    region1 = (a.get("address", {}) or {}).get("region_1depth_name") or \
                              (a.get("road_address", {}) or {}).get("region_1depth_name")
                    x = a.get("x"); y = a.get("y")
                    ra2 = (a.get("road_address", {}) or {}).get("address_name") or ""
                    if region1 == "경기" and x and y and within_rect(float(x), float(y), GG_RECT) and road_pat.search(ra2):
                        return float(x), float(y), {
                            "place_name": q,
                            "address_name": (a.get("address", {}) or {}).get("address_name"),
                            "road_address_name": ra2,
                            "x": x, "y": y
                        }

        # 6) BBOX 확인
        if within_rect(lon, lat, GG_RECT):
            return lon, lat, best

        time.sleep(retry_sleep)

    # 모두 실패
    return None, None, None


# ============================================
# 🚀 5) 1차 실행: 유니크 지명 → 좌표 매핑
# ============================================
unique_names = (
    df["BIZDIST_NM"]
    .astype(str).str.strip()
    .replace({"nan": None})
    .dropna()
    .unique()
)

cache  = load_cache(CACHE_PATH)   # {name: {lon,lat,payload}}
rows   = []
failed = []

try:
    for i, name in enumerate(tqdm(unique_names, desc="Geocoding BIZDIST_NM", unit="name"), start=1):
        # 캐시 히트
        if name in cache and cache[name].get("lon") is not None and cache[name].get("lat") is not None:
            item = cache[name]
            lon, lat, payload = item["lon"], item["lat"], item.get("payload")
        else:
            lon, lat, payload = geocode_one(name)
            cache[name] = {"lon": lon, "lat": lat, "payload": payload}
            time.sleep(SLEEP_SEC)

        rows.append({"BIZDIST_NM": name, "lon": lon, "lat": lat, "meta": payload})
        if (lon is None) or (lat is None):
            failed.append(name)

        if i % CHECKPOINT_EVERY == 0:
            pd.DataFrame(rows).to_csv(CHECKPOINT_PATH, index=False, encoding="utf-8-sig")
            save_cache(CACHE_PATH, cache)

except KeyboardInterrupt:
    print("\n⛔️ 중단 감지: 진행분 저장 중...")

finally:
    map_df = pd.DataFrame(rows)
    map_df.to_csv(CHECKPOINT_PATH, index=False, encoding="utf-8-sig")
    save_cache(CACHE_PATH, cache)
    if failed:
        pd.DataFrame({"BIZDIST_NM": sorted(set(failed))}).to_csv(FAILED_PATH, index=False, encoding="utf-8-sig")
        print(f"❌ 1차 실패: {len(set(failed))}개 → {FAILED_PATH} 저장")

# 병합(원본 df에 lon/lat/meta 추가)
df = df.merge(map_df, on="BIZDIST_NM", how="left")
print(f"✅ 1차 병합 완료: 총 {len(df)}행 / 좌표성공 {df['lon'].notna().sum()}행 ({df['lon'].notna().mean():.1%})")


# ============================================
# 🔁 6) 2차: 실패목록 재시도(보정 포함 geocode_one 사용)
# ============================================
def retry_failed_names(failed_csv=FAILED_PATH, sleep=0.35):
    if not os.path.exists(failed_csv):
        print("⏭ 재시도 스킵: 실패 목록 파일이 없습니다.")
        return pd.DataFrame()
    fnames = pd.read_csv(failed_csv, encoding="utf-8-sig")["BIZDIST_NM"].astype(str).tolist()
    rows2, still = [], []
    for nm in tqdm(fnames, desc="Retry failed", unit="name"):
        lon, lat, payload = geocode_one(nm, retry_sleep=sleep)
        if lon is not None and lat is not None:
            rows2.append({"BIZDIST_NM": nm, "lon": lon, "lat": lat, "meta": payload})
        else:
            still.append(nm)
    fixed_df = pd.DataFrame(rows2)
    if not fixed_df.empty:
        fixed_df.to_csv("failed_fixed.csv", index=False, encoding="utf-8-sig")
    if still:
        pd.DataFrame({"BIZDIST_NM": still}).to_csv("failed_still.csv", index=False, encoding="utf-8-sig")
    print(f"🔁 재시도 결과: 고쳐짐 {len(rows2)} / 여전히 실패 {len(still)}")
    return fixed_df

fixed_df = retry_failed_names(FAILED_PATH, sleep=0.35)

# 고쳐진 결과를 df에 덮어쓰기
if not fixed_df.empty:
    # 우선 map_df도 업데이트(로그/재사용 위해)
    map_df_fixed = map_df.drop(columns=["lon","lat","meta"], errors="ignore") \
                         .merge(fixed_df, on="BIZDIST_NM", how="left", suffixes=("","_new"))
    # 원래 성공값은 보존, NaN만 채우기
    map_df["lon"]  = map_df["lon"].fillna(map_df_fixed["lon"])
    map_df["lat"]  = map_df["lat"].fillna(map_df_fixed["lat"])
    map_df["meta"] = map_df["meta"].where(map_df["meta"].notna(), map_df_fixed["meta"])

    # 원본 df 갱신
    df = df.drop(columns=["lon","lat","meta"], errors="ignore") \
           .merge(map_df, on="BIZDIST_NM", how="left")

print(f"🏁 최종 좌표성공 {df['lon'].notna().sum()} / {len(df)} ({df['lon'].notna().mean():.1%})")

# (선택) 최종 저장
df.to_csv("경기도골목상권매출_위경도(1).csv", index=False, encoding="utf-8-sig")
print("💾 저장: 경기도골목상권매출_위경도(1).csv  /  진행스냅샷:", CHECKPOINT_PATH, " / 캐시:", CACHE_PATH)


Geocoding BIZDIST_NM: 100%|██████████| 1640/1640 [04:06<00:00,  6.66name/s]


❌ 1차 실패: 115개 → failed_names.csv 저장
✅ 1차 병합 완료: 총 69194행 / 좌표성공 64258행 (92.9%)


Retry failed: 100%|██████████| 115/115 [04:06<00:00,  2.15s/name]


🔁 재시도 결과: 고쳐짐 0 / 여전히 실패 115
🏁 최종 좌표성공 64258 / 69194 (92.9%)
💾 저장: BIZDIST_with_coords.csv  /  진행스냅샷: map_progress.csv  / 캐시: map_cache.json


In [34]:
df

,STD_YY,QU_NM,DIV,BIZDIST_NM,CLASS_CD,CLASS_CD_NM,AMT,NOC,lon,lat,meta
0,2024,3,1143,여양로,47591,전기용품 및 조명장치 소매업,2352809,36,127.605348,37.356399,"{'address_name': '경기 여주시 대신면 후포리 500-13', 'cat..."
1,2024,3,1143,여양로,47592,"주방용품 및 가정용 유리, 요업제품 소매업",6969499,5,127.605348,37.356399,"{'address_name': '경기 여주시 대신면 후포리 500-13', 'cat..."
2,2024,3,1143,여양로,47611,"서적, 신문 및 잡지류 소매업",45261,1,127.605348,37.356399,"{'address_name': '경기 여주시 대신면 후포리 500-13', 'cat..."
3,2024,3,1143,여양로,47631,운동 및 경기용품 소매업,5416628,20,127.605348,37.356399,"{'address_name': '경기 여주시 대신면 후포리 500-13', 'cat..."
4,2024,3,1143,여양로,47632,자전거 및 기타 운송장비 소매업,4762640,90,127.605348,37.356399,"{'address_name': '경기 여주시 대신면 후포리 500-13', 'cat..."
...,...,...,...,...,...,...,...,...,...,...,...
69189,2024,3,1648,올레플라자시흥점,47591,전기용품 및 조명장치 소매업,12616929,202,126.798050,37.444513,"{'address_name': '경기 시흥시 은행동 535', 'category_g..."
69190,2024,3,1648,올레플라자시흥점,47520,가구 소매업,2032093,1,126.798050,37.444513,"{'address_name': '경기 시흥시 은행동 535', 'category_g..."
69191,2024,3,1648,올레플라자시흥점,47511,철물 및 난방용구 소매업,62150740,239,126.798050,37.444513,"{'address_name': '경기 시흥시 은행동 535', 'category_g..."
69192,2024,3,1648,올레플라자시흥점,47422,의복 액세서리 및 모조 장신구 소매업,33922877,522,126.798050,37.444513,"{'address_name': '경기 시흥시 은행동 535', 'category_g..."


### 실패한 것들 재시도 

In [35]:
# ===== 1) 경기도 시/군/구 목록 =====
GG_CITIES = [
    # 시(광역 대도시 우선)
    "수원시", "용인시", "고양시", "성남시", "부천시", "안산시", "안양시", "남양주시",
    "화성시", "평택시", "의정부시", "시흥시", "파주시", "김포시", "광명시", "광주시",
    "군포시", "구리시", "오산시", "이천시", "하남시", "의왕시", "안성시", "양주시",
    "동두천시", "과천시", "여주시", "포천시",
    # 군
    "양평군", "가평군", "연천군"
]

# 이름 속에서 이미 '수원','의정부' 같은 시/군 키워드가 보이면 그걸 최우선으로 시도
def _extract_city_hint_from_name(name: str) -> list[str]:
    hits = []
    for c in GG_CITIES:
        short = c.replace("시","").replace("군","")
        if short and short in name:
            hits.append(c)
    return list(dict.fromkeys(hits))  # 중복 제거, 순서 유지

# '파주금촌동우체국' → '파주 금촌동 우체국' 같이 행정단위 분리 강화
def _split_admin_units(s: str) -> str:
    t = s
    # (시/군) + (동/읍/면/리) + 시설명
    t = re.sub(r"^([가-힣]+시)([가-힣]+(동|읍|면|리))우체국$", r"\1 \2 우체국", t)
    t = re.sub(r"^([가-힣]+시)([가-힣]+(동|읍|면|리))우편취급국$", r"\1 \2 우편취급국", t)
    t = re.sub(r"^([가-힣]+시)([가-힣]+(동|읍|면|리))정류장$", r"\1 \2 정류장", t)
    t = re.sub(r"^([가-힣]+시)([가-힣]+(동|읍|면|리))버스터미널$", r"\1 \2 버스터미널", t)
    # 군 단위도 처리
    t = re.sub(r"^([가-힣]+군)([가-힣]+(면|리))우체국$", r"\1 \2 우체국", t)
    t = re.sub(r"^([가-힣]+군)([가-힣]+(면|리))우편취급국$", r"\1 \2 우편취급국", t)
    return t

# 도로명 ‘보광로’ 같이 짧은 이름은 시 힌트를 강제 부여한 쿼리 생성
def build_city_sweep_queries(raw_name: str) -> list[str]:
    name = raw_name.strip()
    qs = []

    # 0) 원본/정규화(네가 이미 쓰는 normalize_candidates) 기반 베이스 확보
    base_cands = normalize_candidates(name)
    base_cands = [_split_admin_units(x) for x in base_cands]

    # 1) 이름 안에 이미 보이는 시/군 힌트를 최우선으로
    seen_hints = _extract_city_hint_from_name(" ".join(base_cands))
    if seen_hints:
        for city in seen_hints:
            for b in base_cands:
                qs.append(f"{city} {b}")

    # 2) 우체국/정류장/버스터미널/역출구 케이스는 도시 힌트가 결정타
    is_poi = bool(re.search(r"우체국|우편취급국|취급국|정류장|버스터미널|터미널|역(\s*\d+번출구)?$", name))
    is_road = _is_road_like(name) or bool(re.search(r"\d+번길$", name))

    # 3) 도로명 짧음(보광로 등) 또는 POI면 도시 스윕 확대
    city_pool = GG_CITIES
    if is_road or is_poi or len(_normalize_text(name)) <= 6:
        for city in city_pool:
            for b in base_cands:
                qs.append(f"{city} {b}")

    # 4) ‘경기도 {city} {name}’ 강제 버전도 추가
    for city in city_pool[:12]:  # 상위 도시 12개만 우선(속도/쿼터 보호)
        for b in base_cands[:3]: # 후보 3개만
            qs.append(f"경기도 {city} {b}")

    # 5) 중복 제거 및 과도한 길이 컷
    qset, out = set(), []
    for q in qs:
        q = re.sub(r"\s+", " ", q).strip()
        if 1 < len(q) < 60 and q not in qset:
            qset.add(q); out.append(q)
    return out[:120]  # 최종 120개 이내로 제한


In [36]:
def geocode_with_city_sweep(name: str, sleep=0.25):
    # 1차: 기존 geocode_one (이미 실행했으므로 보통 실패 케이스만 옴)
    lon, lat, payload = geocode_one(name, retry_sleep=sleep)
    if lon is not None and lat is not None:
        return lon, lat, payload, name

    # 2차: 도시 스윕 쿼리들 순차 시도
    for q in build_city_sweep_queries(name):
        lon, lat, payload = geocode_one(q, retry_sleep=sleep)
        if lon is not None and lat is not None:
            return lon, lat, payload, q
    return None, None, None, None


def retry_failed_with_city_sweep(failed_csv="failed_names.csv", sleep=0.30):
    if not os.path.exists(failed_csv):
        print("⏭ 실패 목록이 없습니다."); 
        return pd.DataFrame(), []

    fnames = pd.read_csv(failed_csv, encoding="utf-8-sig")["BIZDIST_NM"].astype(str).tolist()
    fixed_rows, still = [], []

    for nm in tqdm(fnames, desc="City-sweep retry", unit="name"):
        lon, lat, payload, used_q = geocode_with_city_sweep(nm, sleep=sleep)
        if lon is not None and lat is not None:
            fixed_rows.append({
                "BIZDIST_NM": nm,        # 원래 이름
                "used_query": used_q,    # 어떤 쿼리로 성공했는지
                "lon": lon, "lat": lat, "meta": payload
            })
        else:
            still.append(nm)

    fixed_df = pd.DataFrame(fixed_rows)
    if not fixed_df.empty:
        fixed_df.to_csv("failed_fixed_city.csv", index=False, encoding="utf-8-sig")
    if still:
        pd.DataFrame({"BIZDIST_NM": still}).to_csv("failed_still_city.csv", index=False, encoding="utf-8-sig")
    print(f"🏙️ 도시스윕 결과: 고쳐짐 {len(fixed_rows)} / 여전히 실패 {len(still)}")
    return fixed_df, still


In [ ]:
fixed_city_df, still = retry_failed_with_city_sweep("failed_names.csv", sleep=0.35)

if not fixed_city_df.empty:
    # map_df 업데이트 (있다고 가정)
    base = map_df.drop(columns=["lon","lat","meta"], errors="ignore")
    patched = base.merge(fixed_city_df[["BIZDIST_NM","lon","lat","meta"]], on="BIZDIST_NM", how="left")
    map_df["lon"]  = map_df["lon"].fillna(patched["lon"])
    map_df["lat"]  = map_df["lat"].fillna(patched["lat"])
    map_df["meta"] = map_df["meta"].where(map_df["meta"].notna(), patched["meta"])

    # 원본 df에도 반영
    df = df.drop(columns=["lon","lat","meta"], errors="ignore").merge(map_df, on="BIZDIST_NM", how="left")

    print(f"✅ 보강 병합 완료: 좌표성공 {df['lon'].notna().sum()} / {len(df)} ({df['lon'].notna().mean():.1%})")
    df.to_csv("경기도골목상권매출_위경도(2).csv", index=False, encoding="utf-8-sig")
    print("💾 저장: 경기도골목상권매출_위경도(2).csv")
else:
    print("❗보강 성공 항목이 없습니다. failed_still_city.csv 확인 후 ALIAS/정규화 규칙 추가 권장.")


City-sweep retry: 100%|██████████| 115/115 [5:52:21<00:00, 183.84s/name] 


🏙️ 도시스윕 결과: 고쳐짐 11 / 여전히 실패 104
✅ 보강 병합 완료: 좌표성공 64623 / 69194 (93.4%)
💾 저장: BIZDIST_with_coords_v2.csv
